In [1]:
# ==============
# Import modules
# ==============
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io
import os
import seaborn as sns
from matplotlib.colors import ListedColormap

# Changing the CWD
os.chdir('/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_helper')
from F0AM_reader_MASTER import *
from data_processing import *
from Flight_transect_csv_reader import *
from Plotting_helper import *
warnings.filterwarnings('ignore')

In [2]:
def get_ROx_self_removals(df_loss_raw):
    """
    Filters out specific types of reactions from the DataFrame.

    :param df_loss_raw: DataFrame with reaction names as columns.
    :return: DataFrame filtered to exclude specific reactions.
    """
    ROx_removals = []
    for col_name in df_loss_raw.columns:
        col_name_dummy = col_name.replace(' {+M}', '')
        if ' + ' in col_name: 
            compounds = col_name_dummy.split(' + ')
        else:
            compounds = [col_name_dummy]
        # Filter out reactions having NO and NO2
        if 'NO2' in compounds or 'NO' in compounds:
            continue
        # Filter out OH + VOCs (excluding reactions where OH reacts only with HO2 or itself)
        if 'OH' in compounds and not (len(set(compounds)) == 1 or 'HO2' in compounds):
            continue
        # Filter out NO3
        if 'NO3' in compounds:
            continue
        # Filter out Cl and ClO + HO2
        if 'Cl' in compounds or 'ClO' in compounds:
            continue
            
        # Filter out CH3SOO2 and CH3SO2O2
        if 'CH3SOO2' in compounds or 'CH3SO2O2' in compounds:
            continue
        
        # Combine the data
        ROx_removals.append(col_name)
    return df_loss_raw[ROx_removals]

In [3]:
def read_F0AM_rates(file_name, topX):
    # 1)define the F0AM data stuc nam
    data_struc = 'SpRates'
    # 2) load file
    mod_struc = scipy.io.loadmat(file_name)
    # 3) get data from model
    Time_raw = mod_struc['Time']
    Pnames = mod_struc['Pnames']
    Prod  = mod_struc['Prod']
    Lnames = mod_struc['Lnames']
    Loss  = mod_struc['Loss']
    # 4) flatten the data, i.e., names
    Time = np.array([x[0] for x in Time_raw])/60.0 #/60.0 # the time is in unit of hour
    Pnames = flatten_rate_names(Pnames)
    if 'Dilution' in Lnames: 
        Lnames = flatten_rate_names(Lnames)[:-1] # ignore dilution as this is not considered    
    else:
        Lnames = flatten_rate_names(Lnames) 
    # Work around 
    if len(Loss[0]) - len(Lnames) == 1: 
        Loss = [row[1:] for row in Loss]
    # 5) create dataframe
    df_prod_raw = pd.DataFrame(Prod, columns = Pnames)
    df_loss_raw = pd.DataFrame(Loss, columns = Lnames)
    # 5.5) select the top X
    df_prod = get_mean_topX_df(df_prod_raw, topX, ascending=False)
    df_loss = get_mean_topX_df(df_loss_raw, topX, ascending=True)
    # 6) make the dindex as time
    df_prod.index = Time
    df_loss.index = Time
    # 7) get the data structure and return it
        
    df_strc = {'Prod': df_prod.iloc[1:],
               'Loss': df_loss.iloc[1:]}
    
    return df_strc

In [6]:
# ---------------
# Plot parameters
# ---------------
# Read compounds and each flight dataframe
Flight_IDs   = ['P-3B', 'RF03', 'RF07', 'RF09', 'FN19']
#Flight_IDs   = ['RF09']

compounds = ['ROx_broad', 'NOx_broad', 'NOx', 'HNO3', 'HNO4', 
             'PNs', 'ANs' , 'PAN',
             'ROx', 'RO2', 'CH3CO3',
             'O3', 'Ox',
             'HCOOH', 'MEK']


compounds = ['NOx_broad', 'NOx', 'HNO3', 'HNO4', 
             'PNs', 'ANs' , 'PAN',
             'ROx', 'RO2', 'CH3CO3',
             'O3', 'Ox',
             'HCOOH', 'MEK']

#compounds = ['NOx_broad']


# Initialize an empty DataFrame to store median production rates
df_prod_save = pd.DataFrame(index=compounds, columns=Flight_IDs)
df_loss_save = pd.DataFrame(index=compounds, columns=Flight_IDs)

# Prefix of save-out directory
sav_prefix = '/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/PL_budget/output/'

for row, compound in enumerate(compounds):
    for col, Flight_ID in enumerate(Flight_IDs):
        print(Flight_ID)
        # ------------
        # Reading data
        # ------------
        # Get initialization data from observations
        # Auto change for each flight ID
        if Flight_ID in ['RF03', 'RF07', 'RF09']: file_prefix_obs  = '/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_analysis_TS/WE-CAN/Dataprocess/analysis_bycompound/'
        if Flight_ID in ['FN19']: file_prefix_obs = '/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_analysis_TS/FIREX-AQ/Dataprocess/analysis_bycompound/'
        if Flight_ID in ['P-3B']: file_prefix_obs = '/glade/work/lixujin/PYTHON/SciProj/Box_modeling_analysis/F0AM_analysis_TS/P-3B/Dataprocess/analysis_bycompound/'
    
        df_dummy_obs = pd.read_csv(f'{file_prefix_obs}O3/{Flight_ID}_obs_smk.csv', index_col=0)
        init_time    = df_dummy_obs.index[0]
    
        # Auto change for each flight ID
        if Flight_ID in ['RF03', 'RF07', 'RF09']: file_prefix  = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/WE-CAN/output_data/'
        if Flight_ID in ['FN19']: file_prefix = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/FIREX-AQ/output_data/'
        if Flight_ID in ['P-3B']: file_prefix = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/P-3B/output_data/'    
        # Model files
        file_name_gc = file_prefix + 'pltrate_' + compound + '_rates_GCv13_base' + Flight_ID.replace('-', '') + '.mat'
        file_name_mcmbbvoc= file_prefix + 'pltrate_' + compound + '_rates_MCMv331_base' + Flight_ID.replace('-', '')  + '.mat'
        file_name_mcmgcvoc = file_prefix + 'pltrate_' + compound + '_rates_MCMv331_GCvocs' + Flight_ID.replace('-', '')  + '.mat'

        # Read production and loss rate data
        df_Prod_mcmbbvoc = read_F0AM_rates(file_name_mcmbbvoc, 1000)['Prod']
        df_Prod_mcmgcvoc = read_F0AM_rates(file_name_mcmgcvoc, 1000)['Prod']
        df_Prod_gc       = read_F0AM_rates(file_name_gc, 1000)['Prod']
        
        if compound != 'ROx':
            df_Loss_mcmbbvoc = read_F0AM_rates(file_name_mcmbbvoc, 1000)['Loss']
            df_Loss_mcmgcvoc = read_F0AM_rates(file_name_mcmgcvoc, 1000)['Loss']
            df_Loss_gc       = read_F0AM_rates(file_name_gc, 1000)['Loss']
        else:
            df_Loss_mcmbbvoc = get_ROx_self_removals(read_F0AM_rates(file_name_mcmbbvoc, 1000)['Loss'])
            df_Loss_mcmgcvoc = get_ROx_self_removals(read_F0AM_rates(file_name_mcmgcvoc, 1000)['Loss'])
            df_Loss_gc       = get_ROx_self_removals(read_F0AM_rates(file_name_gc, 1000)['Loss'])

        # Modify the model index
        df_Prod_mcmbbvoc.index = df_Prod_mcmbbvoc.index + init_time
        df_Prod_mcmgcvoc.index = df_Prod_mcmgcvoc.index + init_time
        df_Prod_gc.index       = df_Prod_gc.index + init_time
        df_Loss_mcmbbvoc.index = df_Loss_mcmbbvoc.index + init_time
        df_Loss_mcmgcvoc.index = df_Loss_mcmgcvoc.index + init_time
        df_Loss_gc.index       = df_Loss_gc.index + init_time
        
        # Standardize the model simulations
        standard_index   = df_Prod_mcmbbvoc.index
        df_Prod_mcmgcvoc = interp(df_Prod_mcmgcvoc, standard_index)
        df_Prod_gc       = interp(df_Prod_gc, standard_index)
        df_Loss_mcmgcvoc = interp(df_Loss_mcmgcvoc, standard_index)
        df_Loss_gc       = interp(df_Loss_gc, standard_index)
        
        
        #if compound in ['O3', 'Ox']:
        #    # Need extra step to remove ANs from the cycle.
        #    df_Loss_mcmgcvoc = O3_PL_budget_processor(df_Loss_mcmgcvoc)
        #    df_Loss_gc       = O3_PL_budget_processor(df_Loss_gc)
        #    df_Loss_mcmbbvoc = O3_PL_budget_processor(df_Loss_mcmbbvoc)
        
        # Get the time evolution of total rates
        df_Prod_mcmbbvoc_evolution  = df_Prod_mcmbbvoc.sum(axis=1)
        df_Prod_mcmgcvoc_evolution  = df_Prod_mcmgcvoc.sum(axis=1)
        df_Prod_gc_evolution    = df_Prod_gc.sum(axis=1)
        
        df_Loss_mcmbbvoc_evolution  = df_Loss_mcmbbvoc.sum(axis=1)
        df_Loss_mcmgcvoc_evolution  = df_Loss_mcmgcvoc.sum(axis=1)
        df_Loss_gc_evolution        = df_Loss_gc.sum(axis=1)
        

        # Get the plume-average rates for each reaction
        df_Prod_mcmbbvoc_plume_avg = df_Prod_mcmbbvoc.sum(axis=0)
        df_Prod_mcmgcvoc_plume_avg = df_Prod_mcmgcvoc.sum(axis=0)
        df_Prod_gc_plume_avg       = df_Prod_gc.sum(axis=0)
        
        df_Loss_mcmbbvoc_plume_avg = df_Loss_mcmbbvoc.sum(axis=0)
        df_Loss_mcmgcvoc_plume_avg = df_Loss_mcmgcvoc.sum(axis=0)
        df_Loss_gc_plume_avg       = df_Loss_gc.sum(axis=0)
        
        
        #print(df_Loss_gc_plume_avg[-5:])
        #print(df_Loss_mcmgcvoc_plume_avg[-5:])
        
        # Put data in the dictionary
        data_Prod_evolution = {
            'GEOS-Chem (base)': df_Prod_gc_evolution,
            'MCM + FUR': df_Prod_mcmbbvoc_evolution,
            'MCM + GEOS-Chem VOCs': df_Prod_mcmgcvoc_evolution
        }
        
        data_Prod_plume_avg = {
            'GEOS-Chem (base)': df_Prod_gc_plume_avg,
            'MCM + FUR': df_Prod_mcmbbvoc_plume_avg,
            'MCM + GEOS-Chem VOCs': df_Prod_mcmgcvoc_plume_avg
        }
    
        data_Loss_evolution  = {
            'GEOS-Chem (base)': df_Loss_gc_evolution,
            'MCM + FUR': df_Loss_mcmbbvoc_evolution,
            'MCM + GEOS-Chem VOCs': df_Loss_mcmgcvoc_evolution
        }    
        data_Loss_plume_avg  = {
            'GEOS-Chem (base)': df_Loss_gc_plume_avg,
            'MCM + FUR': df_Loss_mcmbbvoc_plume_avg,
            'MCM + GEOS-Chem VOCs': df_Loss_mcmgcvoc_plume_avg
        }
        
        # Create a DataFrame from the dictionary
        df_Prod_plume_avg  = pd.DataFrame(data_Prod_plume_avg)
        df_Prod_evolution  = pd.DataFrame(data_Prod_evolution)
        df_Loss_plume_avg  = pd.DataFrame(data_Loss_plume_avg)
        df_Loss_evolution  = pd.DataFrame(data_Loss_evolution)
        
        # ---------
        # Save data
        # ---------
        try:
            os.mkdir(f'{sav_prefix}{compound}')
            print("Directory '%s' created" %compound)
        except:
            print("Directory '%s' already exist" %compound)
        

        df_Prod_mcmbbvoc.to_csv(f'{sav_prefix}{compound}/{Flight_ID}_Production_rates_full_MCMBBVOC.csv')    
        df_Loss_mcmbbvoc.to_csv(f'{sav_prefix}{compound}/{Flight_ID}_Loss_rates_full_MCMBBVOC.csv')    

        df_Prod_mcmgcvoc.to_csv(f'{sav_prefix}{compound}/{Flight_ID}_Production_rates_full_MCMGCVOC.csv') 
        df_Loss_mcmgcvoc.to_csv(f'{sav_prefix}{compound}/{Flight_ID}_Loss_rates_full_MCMGCVOC.csv')    
        
        df_Prod_gc.to_csv(f'{sav_prefix}{compound}/{Flight_ID}_Production_rates_full_GC.csv') 
        df_Loss_gc.to_csv(f'{sav_prefix}{compound}/{Flight_ID}_Loss_rates_full_GC.csv')    
        
        df_Prod_evolution.to_csv(f'{sav_prefix}{compound}/{Flight_ID}_Production_rates_evolution.csv')        
        df_Prod_plume_avg.to_csv(f'{sav_prefix}{compound}/{Flight_ID}_Production_rates_plume_avg.csv')
        df_Loss_evolution.to_csv(f'{sav_prefix}{compound}/{Flight_ID}_Loss_rates_evolution.csv')
        # comment out for now, Loss rate in different mechanisms could vary
        #df_Loss_plume_avg.to_csv(f'{sav_prefix}{compound}/{Flight_ID}_Loss_rates_plume_avg.csv')

P-3B
Directory 'NOx_broad' already exist
RF03
Directory 'NOx_broad' already exist
RF07
Directory 'NOx_broad' already exist
RF09
Directory 'NOx_broad' already exist
FN19
Directory 'NOx_broad' already exist
P-3B
Directory 'NOx' already exist
RF03
Directory 'NOx' already exist
RF07
Directory 'NOx' already exist
RF09
Directory 'NOx' already exist
FN19
Directory 'NOx' already exist
P-3B
Directory 'HNO3' already exist
RF03
Directory 'HNO3' already exist
RF07
Directory 'HNO3' already exist
RF09
Directory 'HNO3' already exist
FN19
Directory 'HNO3' already exist
P-3B
Directory 'HNO4' already exist
RF03
Directory 'HNO4' already exist
RF07
Directory 'HNO4' already exist
RF09
Directory 'HNO4' already exist
FN19
Directory 'HNO4' already exist
P-3B
Directory 'PNs' already exist
RF03
Directory 'PNs' already exist
RF07
Directory 'PNs' already exist
RF09
Directory 'PNs' already exist
FN19
Directory 'PNs' already exist
P-3B
Directory 'ANs' already exist
RF03
Directory 'ANs' already exist
RF07
Directory 

Check reactions of ROx

In [5]:
print('done')

done
